In [13]:
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
from yahoo_fin import stock_info as si

In [30]:
def create_initials():
    inital_allocs = pd.read_csv('assets/init_alloc.csv', index_col='stocks') # read in initial allc csv
    inital_allocs = inital_allocs*1000000 # convert to $ value
    stocks = list(inital_allocs.index) # put all stocks in list
    px_dat = [] # px data for day 0
    for stock in stocks:# loop through all stocks
        k = si.get_data(stock, start_date='2021-12-31', end_date='2022-01-01', interval='1d') # request relevant info
        px = k['close'].iloc[0] # take only the closing px on the day
        px_dat.append(px) # append to the list
    inital_allocs['px_dat'] = px_dat # add px data to alloc df
    init_shares = inital_allocs.floordiv(inital_allocs['px_dat'].to_numpy(), axis=0)

    return init_shares

In [31]:
create_initials()


,Anish,Emily,Paul,Kirby,Hosegood,Leong,Olivia,Brokaw,Eric H,Kyle Cross,...,Farrah,Baldridge,Rubyann,Mac,Madison,Karsa,Sergio,Daniel K,Candice,px_dat
stocks,,,,,,,,,,,,,,,,,,,,,
RIVN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,964.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
LCID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2628.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
APPS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1639.0,0.0,4099.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
BUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9469.0,0.0,4734.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
FIGS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3628.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIDU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,672.0,1.0
BABA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,841.0,1.0
DIDI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20080.0,1.0


In [43]:
si.get_data('AAPL', start_date='2021-12-31', end_date='2022-01-03', interval='1d')

,open,high,low,close,adjclose,volume,ticker
2021-12-31,178.089996,179.229996,177.259995,177.570007,177.344055,64062300,AAPL


In [6]:
inital_allocs = pd.read_csv('assets/init_alloc.csv', index_col='stocks')
inital_allocs = inital_allocs*1000000

In [7]:
inital_allocs

,Anish,Emily,Paul,Kirby,Hosegood,Leong,Olivia,Brokaw,Eric H,Kyle Cross,...,Westy,Farrah,Baldridge,Rubyann,Mac,Madison,Karsa,Sergio,Daniel K,Candice
stocks,,,,,,,,,,,,,,,,,,,,,
RIVN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LCID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
APPS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0,0.0,250000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0,0.0,50000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FIGS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIDU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0
BABA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0
DIDI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0
